# Bigram Language Model

This notebook implements a classic statistical Bigram Language Model from scratch.

### 1. Imports

First, we import the necessary libraries. `collections` is used for efficiently counting word occurrences.

In [1]:
import random
from collections import defaultdict, Counter
import re
import string

Download tiny Shakespeare dataset

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-24 13:05:19--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-06-24 13:05:20 (17.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



### 2. Prepare the tables

In [3]:
def remove_punctuation(s):
    """
    Removes all punctuation from a string and converts it to lowercase.
    """
    return s.lower().translate(str.maketrans('', '', string.punctuation))

In [4]:
remove_punctuation("Hello. How are you?")

'hello how are you'

In [8]:
bigram_counts = defaultdict(Counter)
unigram_counts = Counter()
vocabulary = set()
tokens = []
for line in open("input.txt", encoding='utf-8'):
    sentence = remove_punctuation(line.rstrip()).split()
    tokens.extend(["[START]"] + sentence + ["[END]"])


unigram_counts = Counter(tokens)
vocabulary = set(tokens)

for i in range(len(tokens) - 1):
    prev_word, current_word = tokens[i], tokens[i+1]
    bigram_counts[prev_word][current_word] += 1

### 3. Inspect the Trained Model

Let's look at the vocabulary and the learned bigram counts for a sample word.

In [9]:
print("--- Model Vocabulary ---")
print(sorted(list(vocabulary)))
print("\n--- Bigram Counts (sample for 'i') ---")
print(bigram_counts['i'])

--- Model Vocabulary ---
['3', '[END]', '[START]', 'a', 'abandond', 'abase', 'abate', 'abated', 'abbey', 'abbot', 'abed', 'abels', 'abet', 'abhor', 'abhorrd', 'abhorred', 'abhorring', 'abhors', 'abhorson', 'abide', 'abides', 'abilities', 'ability', 'abilitys', 'abject', 'abjects', 'abjured', 'able', 'ableeding', 'aboard', 'abode', 'abodements', 'aboding', 'abominable', 'abortive', 'abound', 'aboundst', 'about', 'above', 'abraham', 'abrahams', 'abreast', 'abroach', 'abroad', 'absence', 'absent', 'absolute', 'absolutely', 'absolved', 'abstains', 'abstinence', 'abstract', 'abundance', 'abundant', 'abundantly', 'abuse', 'abused', 'abuses', 'abusing', 'abysm', 'accent', 'accents', 'accept', 'acceptance', 'access', 'accessary', 'accident', 'accidental', 'accidentally', 'accidents', 'acclamations', 'accommodations', 'accompanied', 'accompany', 'accomplish', 'accomplishd', 'accomplished', 'accompt', 'accord', 'according', 'accordingly', 'accords', 'account', 'accountant', 'accounted', 'account

In [10]:
len(vocabulary)

12850

In [11]:
len(vocabulary) ** 2 / 1e6

165.1225

In [13]:
len(vocabulary) ** 2 * 4  / 1e6 # MB

660.49

In [14]:
bigram_probabilities = {}
for prev_word, bigrams in bigram_counts.items():
    total_count = unigram_counts[prev_word]
    for current_word, count in bigrams.items():
        bigram_probabilities[(prev_word, current_word)] = count / total_count

In [15]:
print(bigram_probabilities['i', "am"])

0.08016333548248442


In [16]:
print(bigram_probabilities['i', "hate"])

0.002364066193853428


In [17]:
print(bigram_probabilities['am', "i"])

0.09846827133479212


### 4. Generate text

In [18]:
def generate_text(vocabulary, bigram_probabilities, max_length=50):
    """
    Generates text using the trained bigram model.
    """
    generated_words = []
    current_word = "[START]"

    for _ in range(max_length):
        words, probabilities = [], []
        for word in vocabulary:
            words.append(word)
            if (current_word, word) not in bigram_probabilities:
                probabilities.append(0)
            else:
                probabilities.append(bigram_probabilities[current_word, word])

        next_word = random.choices(words, weights=probabilities, k=1)[0]

        if next_word == "[END]":
            if generated_words:
                break
            else:
                current_word = "[START]"
                continue
        generated_words.append(next_word)
        current_word = next_word

    text = ' '.join(generated_words)
    if text:
        return text.capitalize()
    return "Could not generate text."

In [21]:
poem = []
for i in range(5):
    generated_sentence = generate_text(vocabulary, bigram_probabilities, max_length=50)
    poem.append(generated_sentence)
print('\n'.join(poem))

This good friend unless he does make all the fight
And these
Coriolanus
How now choplogic what work
Hastings i promise of many goodly day


In [23]:
poem = []
for i in range(25):
    generated_sentence = generate_text(vocabulary, bigram_probabilities, max_length=50)
    poem.append(generated_sentence)
print('\n'.join(poem))

I am too ominous to sleep under your sword and his or that thou do no brother i will in one half by my lord youll not have
Slandering a pin
Was done
Even in bosworth field
Doubtless burgundy
Subject
First provost
Provost
I do confess to many fly ah montague
Most swoln that hath this may hang all i pray you were but lusty and strip myself secure in the very windows that is caius marcius banishment
Well said hermione
Notable
Thrusts and his sake let me
Will
Polixenes
Well
This chamber put on his own doth command no my nose that still in parliament
Upon your grant this outwardsainted deputy
Arithmetic
Tybalt
I bear his eye advance
If she tongue i pray thee and dissolution hangeth over a hall a statesman all the hand was my waters swell before
That at my soul
But with a sin of thy
Queen elizabeth
